In [1]:
import string
import string
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
!pip install emoji==1.4.2
import emoji

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\batyo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\batyo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Clean/Normalize Arabic Text 
# taskeel, longation, ... etc
def clean_str(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','"','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ']
    
    #remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)
    
    #remove longation
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)
    
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    #trim    
    text = text.strip()

    return text

def PreProcessing(text, unk='<UNK>'):

  for i in range(len(text)):

    # remove links
    text[i] = re.sub(r'https?:\/\/.*[\r\n]*', '', text[i], flags=re.MULTILINE)

    # limit repeated emojis to 2
    prev_emoji = None
    cnt = 0
    remove_duplicate_emoji = []
    for c in text[i]:
        if c in emoji.UNICODE_EMOJI['en']:
            if prev_emoji == c and cnt >=2:
                continue
            if prev_emoji != c:
              cnt = 0
            prev_emoji = c
            cnt+=1
        remove_duplicate_emoji.append(c)
    text[i] = "".join(remove_duplicate_emoji)

    # seperate emojis
    text[i] = ''.join((' '+c+' ') if c in emoji.UNICODE_EMOJI['en'] else c for c in text[i])

    # Clean/Normalize Arabic Text
    text[i] = clean_str(text[i]) 

    # remove english words
    text[i] = re.sub(r'\s*[A-Za-z]+\b', '' , text[i])

    # remove numbers
    text[i] = re.sub("\d+", " ", text[i])

    # remove punctuations
    text[i] = text[i].translate(str.maketrans('', '', string.punctuation))

    # tokenize
    text[i] = nltk.tokenize.word_tokenize(text[i])

    # remove stop-words
    stopwords = set(nltk.corpus.stopwords.words("arabic"))
    text[i] = [word for word in text[i] if word not in stopwords]

    # remove emojis
    text[i] = [word for word in text[i] if word not in emoji.UNICODE_EMOJI['en']]

    # remove empty words
    text[i] = [word for word in text[i] if len(word)>2]

    # if the tweet became empty just add add <UNK> token 
    if len(text[i])==0:
      text[i] = [unk]

  return text